In [1]:
import geopandas, folium, sqlite3, shapely.wkt, os 
from shapely.geometry import Polygon, MultiPolygon
import shapely.ops as so
import pandas as pd

os.chdir('..')
db = '____TIF_Calc_DB____.sqlite'

def query_grab(query):
    conn = sqlite3.connect(db)
    c = conn.cursor()
    c.execute(query)
    df = c.fetchall()
    conn.close()
    return df

In [5]:
# Queries
query1_DDA = '''SELECT 
                ParcelDetail.current_taxable AS Taxable,
                ParcelDetail.dda_tifa_code_0 AS Name,
                GEOM_ASG.the_geom AS Shape
              FROM ParcelDetail
              INNER JOIN 
                GEOM_ASG ON ParcelDetail.parcelnumber = GEOM_ASG.PARCELNUM
              WHERE ParcelDetail.TaxYear IS 2018
                AND ParcelDetail.dda_tifa_code_0 LIKE "DDA%";'''

query2_Z100 ='''SELECT 
                ParcelDetail.current_taxable AS Taxable,
                ParcelDetail.dda_tifa_code_0 AS Name,
                GEOM_ASG.the_geom AS Shape
              FROM ParcelDetail
              INNER JOIN 
                GEOM_ASG ON ParcelDetail.parcelnumber = GEOM_ASG.PARCELNUM
              WHERE ParcelDetail.TaxYear IS 2018
                AND ParcelDetail.dda_tifa_code_1 LIKE "Z100%";'''


query11_TIF ='''SELECT 
                ParcelDetail.current_taxable AS Taxable,
                ParcelDetail.dda_tifa_code_0 AS Name,
                GEOM_ASG.the_geom AS Shape
              FROM ParcelDetail
              INNER JOIN 
                GEOM_ASG ON ParcelDetail.parcelnumber = GEOM_ASG.PARCELNUM
              WHERE ParcelDetail.TaxYear IS 2018
                AND ParcelDetail.dda_tifa_code_1 LIKE "Z1%";'''

In [6]:
#Convert results to wkt
DDA_geoms = [shapely.wkt.loads(x[2])[0] for x in query_grab(query1_DDA)]
Z100_geoms = [shapely.wkt.loads(x[2])[0] for x in query_grab(query2_Z100)]
#Exansion1 = [shapely.wkt.loads(x[2])[0] for x in query_grab(query2_Expansion1)]
ALL_TIF_geoms = [shapely.wkt.loads(x[2])[0] for x in query_grab(query11_TIF)]

#Buffer & Dissolve
new_DDA = so.cascaded_union(MultiPolygon(DDA_geoms).buffer(.0002)) # Buffer roughly distance accross street
new_Z100 = so.cascaded_union(MultiPolygon(Z100_geoms).buffer(.0002))
#new_Expansion1 = so.cascaded_union(MultiPolygon(Exansion1).buffer(.0002))
new_AllTIF = so.cascaded_union(MultiPolygon(ALL_TIF_geoms).buffer(.0002))

In [9]:
#Map with Folium
CENTER = [42.3318175,-83.0465565]
m = folium.Map(CENTER, zoom_start=14, tiles='cartodbpositron')


folium.GeoJson(new_DDA,
    name='Entire DDA Area - Millage',
    style_function=lambda poly: {
        'fillColor': 'grey',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.2,
    }).add_to(m)

folium.GeoJson(new_AllTIF,
    name='All TIF Expansions Combined',
    style_function=lambda poly: {
        'fillColor': 'red',
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.4,
    }).add_to(m)

folium.GeoJson(new_Z100,
    name='Initial TIF Development Area',
    style_function=lambda poly: {
        'fillColor': 'green',
        'color': 'black',
        'weight': 1,
        'dashArray': '5, 5',
        'fillOpacity': 0.4,
    }).add_to(m)

folium.LayerControl().add_to(m)
m

In [10]:
m.save("DDA.html")